Read Mesh

In [1]:
from lnas import LnasFormat
import pathlib
import pandas as pd
import numpy as np

# for i in range(47):
#     print(f"Merging case {i}")
#     case_path = pathlib.Path(f"../insight/Docker/local/volume/escalonamento_s1/BL01/Caso_{i}")
for i in ["000", "045", "090"]:
    print(f"Merging case {i}")
    case_path = pathlib.Path(f"../insight/Docker/local/volume/CBCA/BL01/{i}")

    meshes = []
    dfs = []
    last_index = 0

    for sfc_folder in (case_path / "data" / "surfaces").iterdir():
        print(f"Processing sfc {sfc_folder.name}")
        mesh_path = sfc_folder / "cp_dfs" / "mesh.lnas"
        mesh = LnasFormat.from_file(mesh_path)
        mesh.surfaces[sfc_folder.name] = np.arange(
            0, mesh.geometry.triangles.shape[0]
        )
        
        body_hist_series = pd.read_hdf(sfc_folder / "cp_dfs" / "hist_series.triangles.hdf")
        if last_index == 0:
            last_index = body_hist_series.point_index.max()
        else:
            body_hist_series.point_index += last_index + 1
            last_index = body_hist_series.point_index.max()
        meshes.append(mesh)
        dfs.append(body_hist_series)
        
    new_df = pd.concat(dfs).sort_values(
        by=["time_stamp", "point_index"]
    )
    new_df.to_hdf(case_path / "data.merged.h5", key="df", mode="w")
    del new_df
        
    mesh = meshes[0]
    mesh.geometry.triangles = np.flip(mesh.geometry.triangles, axis=1)
    mesh.geometry._full_update()

    for append_mesh in meshes[1:]:
        tri_normals = append_mesh.geometry.normals.copy()
        tri_verts = append_mesh.geometry.triangle_vertices.copy()

        new_verts = tri_verts.copy().reshape(-1, 3)
        new_verts = np.unique(new_verts, axis=0)

        new_triangles = []

        for tri in tri_verts:
            tri_index = []
            for v in tri:
                matches = np.all(new_verts == v, axis=1)
                indices = np.where(matches)
                tri_index.append(indices[0][0])
            tri_index.reverse()
            new_triangles.append(np.array(tri_index))

        new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

        new_tri_indices = np.arange(
            mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
        )
        mesh.surfaces[[k for k in append_mesh.surfaces.keys()][0]] = new_tri_indices

        mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
        mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

        mesh.geometry._update_triangles_vertices()
        mesh.geometry._update_normals()
        mesh.geometry._update_areas()
        mesh.geometry._update_vertices_normals()


    filename = case_path / "geometry.lnas"
    export_path = case_path / "geometry.stl"

    mesh.to_file(filename)
    mesh.export_stl(export_path)

Merging case 000
Processing sfc b3xp
Processing sfc b1xm
Processing sfc b1t
Processing sfc b1p1
Processing sfc b2p2
Processing sfc b1xp1


: 

In [21]:
from lnas import LnasFormat
import pathlib
import pandas as pd
import numpy as np

case_path = pathlib.Path("../insight/Docker/local/volume/escalonamento_s1/BL01/Caso_0")

meshes = []
dfs = []
last_index = 0

for sfc_folder in (case_path / "surfaces").iterdir():
    mesh_path = sfc_folder / "mesh.lnas"
    mesh = LnasFormat.from_file(mesh_path)
    mesh.surfaces[sfc_folder.name] = np.arange(
        0, mesh.geometry.triangles.shape[0]
    )
    
    body_hist_series = pd.read_hdf(sfc_folder / "hist_series.triangles.hdf")
    if last_index == 0:
        last_index = body_hist_series.point_index.max()
    else:
        body_hist_series.point_index += last_index + 1
        last_index = body_hist_series.point_index.max()
    meshes.append(mesh)
    dfs.append(body_hist_series)
    
body_hist_series

,p,point_index,time_stamp
0,-0.000766,4727,26010.457031
1,-0.000474,4728,26010.457031
2,-0.000345,4729,26010.457031
3,-0.000473,4730,26010.457031
4,-0.000548,4731,26010.457031
...,...,...,...
138,-0.001214,4865,128970.460938
139,-0.000358,4866,128970.460938
140,-0.000614,4867,128970.460938
141,-0.001515,4868,128970.460938


Offset body

In [22]:
import numpy as np

mesh = meshes[0]
mesh.geometry.triangles = np.flip(mesh.geometry.triangles, axis=1)
mesh.geometry._full_update()

for append_mesh in meshes[1:]:
    tri_normals = append_mesh.geometry.normals.copy()
    tri_verts = append_mesh.geometry.triangle_vertices.copy()

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_verts = np.unique(new_verts, axis=0)

    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        tri_index.reverse()
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

    new_tri_indices = np.arange(
        mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
    )
    mesh.surfaces[[k for k in append_mesh.surfaces.keys()][0]] = new_tri_indices

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    mesh.geometry._update_vertices_normals()


filename = case_path / "geometry.lnas"
export_path = case_path / "geometry.stl"

mesh.to_file(filename)
mesh.export_stl(export_path)

Join historic series

In [23]:
new_df = pd.concat(dfs).sort_values(
    by=["time_stamp", "point_index"]
)
new_df.to_hdf(case_path / "data.merged.h5", key="df", mode="w")

new_df

,p,point_index,time_stamp
0,-0.006030,0,26010.457031
1,-0.004214,1,26010.457031
2,-0.003402,2,26010.457031
3,-0.005121,3,26010.457031
4,-0.002757,4,26010.457031
...,...,...,...
138,-0.001214,4865,128970.460938
139,-0.000358,4866,128970.460938
140,-0.000614,4867,128970.460938
141,-0.001515,4868,128970.460938
